In [34]:
import cv2
import mediapipe as mp
import numpy as np
import os
import copy
import itertools
import keras
from keras.callbacks import EarlyStopping
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation, BatchNormalization
import pickle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

classList = ['dry', 'healthy', 'sick','Good Morning']
hands = mp.solutions.hands.Hands(static_image_mode=False, max_num_hands=2,
                                 min_detection_confidence=0.5, min_tracking_confidence=0.5)
sequenceLength = 30

def normalizeCoordinates(coords):
    
    baseX = 0
    baseY = 0

    for i, val in enumerate(coords):
        if i == 0:
            baseX = val[0]
            baseY = val[1]

        coords[i][0] = coords[i][0] - baseX
        coords[i][1] = coords[i][1] - baseY 

    coords = list(itertools.chain.from_iterable(coords))

    maxVal = max(list(map(abs, coords)))

    def normalize_(n):
        return n / maxVal

    coords = list(map(normalize_, coords))

    return coords

def skeletonExtraction(path):

    left = []
    right = []
    cap = cv2.VideoCapture(path)

    while True:
        success, img = cap.read()

        if(success == False):

       
            cap.release()
            break
        
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(imgRGB)
        
        hLeft = None
        hRight = None

        if(results.multi_hand_landmarks):
            for idx, handLms in enumerate(results.multi_hand_landmarks):
                
                hand = []
                for id, lm in enumerate(handLms.landmark):
                    h, w, c = img.shape
                    cx, cy = int(lm.x *w), int(lm.y*h)
                    hand.append([cx, cy])

                label = results.multi_handedness[idx].classification[0].label
                

                if(label == 'Left'):
                    hLeft = normalizeCoordinates(hand)
                elif(label == 'Right'):
                    hRight = normalizeCoordinates(hand)
        
        if(hLeft != None):
            left.append(hLeft)
        if(hRight != None):
            right.append(hRight)
    
    countLeft = len(left)
    countRight = len(right)
    windowLeft = max(countLeft/sequenceLength, 1)
    windowRight = max(countRight/sequenceLength, 1)

    finalFeatures = []

    if countLeft < sequenceLength or countRight < sequenceLength:
        return []


    for i in range(0, sequenceLength):
        
        finalFeatures.append(left[int(i * windowLeft)] + right[int(i * windowRight)])

    return np.asarray(finalFeatures)

def inputProcessing(path):
    featuress = skeletonExtraction(path)
    temp = [featuress]

    return np.asarray(temp)

def predict(fp):
    jsonFile = open('my.json', 'r')
    loadedModel = jsonFile.read()
    loadedModel = keras.models.model_from_json(loadedModel)
    loadedModel.load_weights('my.h5')
    loadedModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    featuress = inputProcessing(fp)
    outputs = loadedModel.predict(featuress)
    labels = classList[np.argmax(outputs)]
    print(labels)

In [36]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
from functools import partial
from tkinter import * 
root = tk.Tk()
root.geometry("800x600")
root.configure(bg="yellow")
file_path="C:/Users/91790/Desktop/untitled-video-made-with-clipchamp_4sJmb4vZ.mp4"
#This function should call the Python program that records webcam video using OpenCV.
def record_video():
    os.system('C:/Users/91790/Desktop/record.py')
    
def choose_file():
    file_path = filedialog.askopenfilename(initialdir="D:/isl_projects/datasets/", title="Select a Video File",
                                           filetypes=(("Video files", "*.MOV"),
                                                      ("all files", "*.*")))
    display_video(file_path)
    
def display_video(file_path):
    cap = cv2.VideoCapture(file_path)
    ret, frame = cap.read()
    if not ret:
        return
    h, w, channels = frame.shape
    scale_factor = 0.25  # Reduce the size by 4x
    h = int(h * scale_factor)
    w = int(w * scale_factor)
    video_canvas.config(width=w, height=h)
    video_canvas.pack()
    while ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (w, h))
        img = Image.fromarray(frame)
        img_tk = ImageTk.PhotoImage(image=img)
        video_canvas.img_tk = img_tk
        video_canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)
        root.update()
        ret, frame = cap.read()
    cap.release()

video_canvas = tk.Canvas(root)
video_canvas.pack()

button1 = tk.Button(root, text="Select Video", command=choose_file)
button1.pack()
button2 = tk.Button(root, text="Record Video", command=record_video)
button2.pack(side="bottom")
btn = tk.Button(root, text="Click Me", command=predict(file_path))
btn.pack(side="bottom")
root.mainloop()

1/1 [==============================] - 1s 769ms/step
Good Morning


NameError: name 'labels' is not defined

In [8]:
import tkinter as tk
import customtkinter as ck
import pandas as pd
import numpy as np
import pickle
import mediapipe as mp
import cv2
from PIL import Image,ImageTk
from tkinter import filedialog

window = tk.Tk()
window.geometry("500x700")
window.title("isl")
ck.set_appearance_mode("dark")

wordlabel=ck.CTkLabel(window,height=20,width=120,padx=10,text_color='black',fg_color='blue')
wordlabel.place(x=10,y=1)
wordlabel.configure(text='word')
wordbox=ck.CTkLabel(window,height=20,width=120,padx=10,text_color='black',fg_color='blue')
wordbox.place(x=10,y=41)
wordbox.configure(text='0')

def select_file():
	global file_name
	file_name = filedialog.askopenfilename(initialdir="D:/isl_projects/datasets/", title="Select a Video File",
                                           filetypes=(("Video files", "*.mp4"),
                                                      ("all files", "*.*")))



button = ck.CTkButton(window,text='select_file',command=select_file,height=20,width=120,text_color='black',fg_color='blue')
button.place(x=10,y=550)



window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\softwares\python310.5\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "D:\softwares\python310.5\lib\tkinter\__init__.py", line 839, in callit
    func(*args)
  File "D:\softwares\python310.5\lib\site-packages\customtkinter\windows\widgets\scaling\scaling_tracker.py", line 178, in check_dpi_scaling
    if window.winfo_exists() and not window.state() == "iconic":
  File "D:\softwares\python310.5\lib\tkinter\__init__.py", line 1112, in winfo_exists
    self.tk.call('winfo', 'exists', self._w))
_tkinter.TclError: can't invoke "winfo" command: application has been destroyed
Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\softwares\python310.5\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "D:\softwares\python310.5\lib\tkinter\__init__.py", line 2340, in destroy
    for c in list(self.children.values()): c.destroy()
 

KeyboardInterrupt: 